In [1]:
import pandas as pd
import requests
from lxml import html

In [2]:
#URL contenant la liste des départements et les URL correspondantes
index = 'http://elections.interieur.gouv.fr/legislatives-2017/index.html'

In [3]:
r = requests.get(index)
c = r.content
tree = html.fromstring(c)

In [4]:
#exemple
elt = tree.xpath('//select[@id="listeDpt"]/option[108]')

In [5]:
#L'attribut value contient le bout d'URL qui mène à ce "département" (en l'occurence il s'agit des français expatriés)
print elt[0].get('value')

099/index.html


In [17]:
print elt[0].text

99 - FranÃ§ais Ã©tablis hors de France


In [18]:
#fct qui pour un indice d'option donné (les départements sont stockés dans une liste déroulante d'options) renvoie
#l'url, le numéro de département et le nom du département
def dep_parse(opnb):
    opt = tree.xpath('//select[@id="listeDpt"]/option[%d]' % opnb)[0]
    txt_split = opt.text.split(' - ')
    return [opt.get('value'), txt_split[0], txt_split[1]]

#formation d'une df contenant les départements et les bouts d'URL à utiliser pour consulter leurs résultats de législatives
deps = pd.DataFrame([dep_parse(i) for i in range(2, 109)], columns=['url', 'nb', 'dep'])

In [19]:
deps.head()

,url,nb,dep
0,001/index.html,01,Ain
1,002/index.html,02,Aisne
2,003/index.html,03,Allier
3,004/index.html,04,Alpes-de-Haute-Provence
4,005/index.html,05,Hautes-Alpes


In [20]:
#on utilisera pas index.html, on gardera uniquement le string de code du département. Pour accéder à une circonscription,
#il faudra en effet ajouter son numéro dans l'URL
deps.url = deps.url.apply(lambda x: x.replace('/index.html', ''))

In [21]:
#exemple
r = requests.get('http://elections.interieur.gouv.fr/legislatives-2017/093/09303.html')
c = r.content
tree = html.fromstring(c)

In [30]:
#test
tree.xpath('//table[@class="table table-bordered"][2]/tbody/tr[1]/td[6]/text()')[0]

'Ballotage'

In [39]:
#obtenir le nom des nuances dans le tableau de résultats
def get_nuances(tree):
    result = []
    j = 1
    while True:
        try:
            result.append(tree.xpath('//table[@class="table table-bordered"][2]/tbody/tr[%d]/td[2]/text()' % j)[0])
            j += 1
        except IndexError:
            break
    return result

In [40]:
#obtenir les noms des candidats
def get_candidates(tree):
    result = []
    j = 1
    while True:
        try:
            result.append(tree.xpath('//table[@class="table table-bordered"][2]/tbody/tr[%d]/td[1]/text()' % j)[0])
            j += 1
        except IndexError:
            break
    return result

In [45]:
#obtenir les scores en % des voix exprimées
def get_scores(tree):
    result = []
    j = 1
    while True:
        try:
            result.append(tree.xpath('//table[@class="table table-bordered"][2]/tbody/tr[%d]/td[5]/text()' % j)[0])
            j += 1
        except IndexError:
            break
    return result

In [58]:
#Création de la df qui contient tous les résultats

df = pd.DataFrame({'dep':[],
                   'circ': [],
                      'parti':[],
                      'score':[]})

for k in range(len(deps)):
    i = 1
    while True:
        if i < 10:
            r = requests.get(index[:-10] + deps['url'][k] + '/' + deps['url'][k] + '0%d.html' % i)
        elif i >= 10:
            r = requests.get(index[:-10] + deps['url'][k] + '/' + deps['url'][k] + '%d.html' % i)
        else:
            print 'WTF ?'
        if r.status_code == 404:
            break
        else:
            c = r.content
            tree = html.fromstring(c)
            circdf = pd.DataFrame({'dep': deps['dep'][k],
                                   'numdep': deps['nb'][k],
                              'circ': i,
                              'parti':get_nuances(tree),
                                  'score': get_scores(tree)})
            df = df.append(circdf, ignore_index=True)
            print "Dep %s circ %d done" % (deps['dep'][k], i)
            i += 1

Dep Ain circ 1 done
Dep Ain circ 2 done
Dep Ain circ 3 done
Dep Ain circ 4 done
Dep Ain circ 5 done
Dep Aisne circ 1 done
Dep Aisne circ 2 done
Dep Aisne circ 3 done
Dep Aisne circ 4 done
Dep Aisne circ 5 done
Dep Allier circ 1 done
Dep Allier circ 2 done
Dep Allier circ 3 done
Dep Alpes-de-Haute-Provence circ 1 done
Dep Alpes-de-Haute-Provence circ 2 done
Dep Hautes-Alpes circ 1 done
Dep Hautes-Alpes circ 2 done
Dep Alpes-Maritimes circ 1 done
Dep Alpes-Maritimes circ 2 done
Dep Alpes-Maritimes circ 3 done
Dep Alpes-Maritimes circ 4 done
Dep Alpes-Maritimes circ 5 done
Dep Alpes-Maritimes circ 6 done
Dep Alpes-Maritimes circ 7 done
Dep Alpes-Maritimes circ 8 done
Dep Alpes-Maritimes circ 9 done
Dep Ardèche circ 1 done
Dep Ardèche circ 2 done
Dep Ardèche circ 3 done
Dep Ardennes circ 1 done
Dep Ardennes circ 2 done
Dep Ardennes circ 3 done
Dep Ariège circ 1 done
Dep Ariège circ 2 done
Dep Aube circ 1 done
Dep Aube circ 2 done
Dep Aube circ 3 done
Dep Aude circ 1 done
Dep Aude circ 2 do

In [59]:
df.head()

,circ,dep,numdep,parti,score
0,1.0,Ain,01,MDM,"33,89"
1,1.0,Ain,01,LR,"26,78"
2,1.0,Ain,01,FN,"15,46"
3,1.0,Ain,01,FI,"9,70"
4,1.0,Ain,01,SOC,"9,23"


In [60]:
#Superbe, on va reformat un peu
df.score = df.score.apply(lambda x: x.replace(' ', ''))

In [61]:
df.score = df.score.apply(lambda x: x.replace(',', '.'))

In [62]:
df.score = df.score.apply(float)

In [67]:
df.parti.value_counts()

DIV    1416
ECO     914
EXG     663
FN      571
DVD     563
FI      557
LR      481
REM     468
COM     461
SOC     413
DLF     392
DVG     376
EXD     178
REG     147
UDI     146
MDM      69
RDG      62
Name: parti, dtype: int64

In [68]:
#dico pour attribuer chaque nuance à un groupe politique
d = {'DIV': 'l_autre',
    'ECO': 'l_EG',
    'EXG': 'l_EG',
    'FN': 'l_ED',
    'DVD': 'l_D',
    'FI': 'l_EG',
    'LR': 'l_D',
    'REM': 'l_C',
    'COM': 'l_EG',
    'SOC': 'l_G',
    'DLF': 'l_ED',
    'DVG': 'l_G',
    'EXD': 'l_ED',
    'REG': 'l_autre',
    'UDI': 'l_autre',
    'MDM': 'l_C',
    'RDG': 'l_G'}

In [69]:
#application
df.parti = df.parti.apply(lambda x: d[x])

In [80]:
df.head()

,circ,dep,numdep,parti,score
0,1.0,Ain,01,l_C,33.89
1,1.0,Ain,01,l_D,26.78
2,1.0,Ain,01,l_ED,15.46
3,1.0,Ain,01,l_EG,9.70
4,1.0,Ain,01,l_G,9.23


In [81]:
#à présent on somme tout ça
somme = df.groupby(['numdep', 'circ', 'parti']).sum()

In [82]:
somme.head(10)

score
numdep circ parti         
01     1.0  l_C      33.89
            l_D      26.78
            l_ED     15.46
            l_EG     13.48
            l_G       9.23
            l_autre   1.15
       2.0  l_C      35.90
            l_D      24.52
            l_ED     19.08
            l_EG     19.18

In [83]:
somme.to_csv('./legis_clean/legis2017.csv', encoding='utf-8')